In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline
#os.chdir('/Users/flatironschooldc3/FlatironSchoolRepo/dc-ds-111819/module-1/project/unzipped_data')
os.chdir('/Users/flatironschooldc/Code/LearnCo/dc-ds-111819/module-1/project/data/')
def duplicates_by(df, columns):
    return df[df.duplicated(subset=columns,keep=False)].sort_values(columns)

In [2]:
# Box Office Mojo (BOM) -- title, year (2010-2018), domestic and froeign gross
bom = pd.read_csv("bom.movie_gross.csv")

# IMDB -- title id, original title, genre, year (2010-2018), runtime, avg rating, number of votes
imdbtb = pd.read_csv("imdb.title.basics.csv")
imdbtr = pd.read_csv("imdb.title.ratings.csv")

In [3]:
# Check out BOM dataset
display(bom.head(2))
display(bom.info())

duplicates_by(bom, "title")

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
title             3387 non-null object
studio            3382 non-null object
domestic_gross    3359 non-null float64
foreign_gross     2037 non-null object
year              3387 non-null int64
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


None

,title,studio,domestic_gross,foreign_gross,year
317,Bluebeard,Strand,33500.0,5200,2010
3045,Bluebeard,WGUSA,43100.0,NaN,2017


In [4]:
# Make index title and check
display(bom.set_index("title").head(2))

# Keep change
bom.set_index("title", inplace = True)

,studio,domestic_gross,foreign_gross,year
title,,,,
Toy Story 3,BV,415000000.0,652000000,2010
Alice in Wonderland (2010),BV,334200000.0,691300000,2010


In [5]:
# Check out IMBDTB dataset
display(imdbtb.head(2))

# Make index tconst to prep for merge with other imdb dataset
# Drop duplicate title column, rename title and year columns for consistency, and check
display(imdbtb.set_index("tconst").drop("primary_title", axis=1).rename({"start_year":"year","original_title":"title"}, axis=1).head(2))

# Keep changes
imdbtb = imdbtb.set_index("tconst").drop("primary_title", axis=1).rename({"start_year":"year","original_title":"title"}, axis=1)

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


,title,year,runtime_minutes,genres
tconst,,,,
tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama"
tt0066787,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


In [6]:
# Check out IMBDTR dataset
display(imdbtr.head(2))

# Make index for this set to merge.
display(imdbtr.set_index("tconst").head(2))

# Keep change
imdbtr.set_index("tconst", inplace = True)

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559


,averagerating,numvotes
tconst,,
tt10356526,8.3,31
tt10384606,8.9,559


In [7]:
# Merge imdb datasets on tconst index, preserving all info for genre dataset but only converged ratings info
display(imdbtb.merge(imdbtr, how = "outer", on="tconst", sort=True, indicator=True,
         validate="1:1").head(10))

# Preserve changes as new dataset
imdbm = imdbtb.merge(imdbtr, how = "outer", on="tconst", sort=True, indicator=True,
         validate="1:1")

imdbm._merge.value_counts()

,title,year,runtime_minutes,genres,averagerating,numvotes,_merge
tconst,,,,,,,
tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,both
tt0066787,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,both
tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,both
tt0069204,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,both
tt0100275,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,both
tt0111414,A Thin Life,2018,75.0,Comedy,NaN,NaN,left_only
tt0112502,Bigfoot,2017,NaN,"Horror,Thriller",4.1,32.0,both
tt0137204,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy",8.1,263.0,both
tt0139613,O Silêncio,2012,NaN,"Documentary,History",NaN,NaN,left_only


both          73856
left_only     72288
right_only        0
Name: _merge, dtype: int64

In [8]:
# Clean new dataset, set title as index after finding duplicates

display(imdbm.info())

imdbm.duplicated("title", keep="first").any()
display(imdbm.info())
# Appears there are no duplicates

imdbm.set_index("title").head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 146144 entries, tt0063540 to tt9916754
Data columns (total 7 columns):
title              146123 non-null object
year               146144 non-null int64
runtime_minutes    114405 non-null float64
genres             140736 non-null object
averagerating      73856 non-null float64
numvotes           73856 non-null float64
_merge             146144 non-null category
dtypes: category(1), float64(3), int64(1), object(2)
memory usage: 7.9+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 146144 entries, tt0063540 to tt9916754
Data columns (total 7 columns):
title              146123 non-null object
year               146144 non-null int64
runtime_minutes    114405 non-null float64
genres             140736 non-null object
averagerating      73856 non-null float64
numvotes           73856 non-null float64
_merge             146144 non-null category
dtypes: category(1), float64(3), int64(1), object(2)
memory usage: 7.9+ MB


None

,year,runtime_minutes,genres,averagerating,numvotes,_merge
title,,,,,,
Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,both
Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,both


In [9]:
# Median runtime, rating, and votes for NaN entries
imdbm_rm = imdbm.runtime_minutes.dropna()
display(imdbm_rm.describe())

imdbm_ar = imdbm.averagerating.dropna()
display(imdbm_ar.describe())

imdbm_nv = imdbm.numvotes.dropna()
display(imdbm_nv.describe())

# Runtime median 87 min close to mean 86 min
# Rating median 6.5 close to mean 6.3
# Vote count median 49 far from mean 3524 Use median anyway.

count    114405.000000
mean         86.187247
std         166.360590
min           1.000000
25%          70.000000
50%          87.000000
75%          99.000000
max       51420.000000
Name: runtime_minutes, dtype: float64

count    73856.000000
mean         6.332729
std          1.474978
min          1.000000
25%          5.500000
50%          6.500000
75%          7.400000
max         10.000000
Name: averagerating, dtype: float64

count    7.385600e+04
mean     3.523662e+03
std      3.029402e+04
min      5.000000e+00
25%      1.400000e+01
50%      4.900000e+01
75%      2.820000e+02
max      1.841066e+06
Name: numvotes, dtype: float64

In [10]:
# Check call for median values
display(imdbm_rm.median())
display(imdbm_ar.median())
display(imdbm_nv.median())

# Replace NaN with median values
imdbm.runtime_minutes.fillna(value = imdbm_rm.median(), inplace=True)
imdbm.averagerating.fillna(value = imdbm_ar.median(), inplace=True)
imdbm.numvotes.fillna(value = imdbm_nv.median(), inplace=True)

87.0

6.5

49.0

In [11]:
# Check replaced NaN values
display(imdbm.info())

# Change index to title to merge with BOM dataset
display(imdbm.set_index("title").head(2))

<class 'pandas.core.frame.DataFrame'>
Index: 146144 entries, tt0063540 to tt9916754
Data columns (total 7 columns):
title              146123 non-null object
year               146144 non-null int64
runtime_minutes    146144 non-null float64
genres             140736 non-null object
averagerating      146144 non-null float64
numvotes           146144 non-null float64
_merge             146144 non-null category
dtypes: category(1), float64(3), int64(1), object(2)
memory usage: 7.9+ MB


None

,year,runtime_minutes,genres,averagerating,numvotes,_merge
title,,,,,,
Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,both
Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,both


In [12]:
# Merge imdbm cleaned dataset with bom dataset on title, preserving all data from sets
display(imdbm.merge(bom, how = "outer", on="title", sort=True, indicator="_merge2"))

# Preserve changes as new dataset
imdbom = imdbm.merge(bom, how = "outer", on="title", sort=True, indicator="_merge2")


,title,year_x,runtime_minutes,genres,averagerating,numvotes,_merge,studio,domestic_gross,foreign_gross,year_y,_merge2
0,#1 Serial Killer,2013.0,87.0,Horror,5.6,40.0,both,NaN,NaN,NaN,NaN,left_only
1,#5,2013.0,68.0,"Biography,Comedy,Fantasy",6.8,6.0,both,NaN,NaN,NaN,NaN,left_only
2,#5,2018.0,87.0,Documentary,6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only
3,#50Fathers,2015.0,132.0,Comedy,6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only
4,#66,2015.0,116.0,Action,5.3,18.0,both,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...
147362,NaN,2018.0,87.0,Adventure,6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only
147363,NaN,2020.0,87.0,NaN,6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only
147364,NaN,2017.0,125.0,Romance,6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only
147365,NaN,2017.0,87.0,NaN,6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only


In [13]:
# Self-check
imdbom.info()
imdbom._merge2.value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147367 entries, 0 to 147366
Data columns (total 12 columns):
title              147346 non-null object
year_x             146144 non-null float64
runtime_minutes    146144 non-null float64
genres             140736 non-null object
averagerating      146144 non-null float64
numvotes           146144 non-null float64
_merge             146144 non-null category
studio             3994 non-null object
domestic_gross     3964 non-null float64
foreign_gross      2431 non-null object
year_y             3999 non-null float64
_merge2            147367 non-null category
dtypes: category(2), float64(6), object(4)
memory usage: 12.6+ MB


left_only     143368
both            2776
right_only      1223
Name: _merge2, dtype: int64

In [14]:
# Because new dataset has such a high proportion of imdbm rather than bom, fill in median values for rating, numvotes, runtime, as before
imdbom.runtime_minutes.fillna(value = imdbm_rm.median(), inplace=True)
imdbom.averagerating.fillna(value = imdbm_ar.median(), inplace=True)
imdbom.numvotes.fillna(value = imdbm_nv.median(), inplace=True)
imdbom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147367 entries, 0 to 147366
Data columns (total 12 columns):
title              147346 non-null object
year_x             146144 non-null float64
runtime_minutes    147367 non-null float64
genres             140736 non-null object
averagerating      147367 non-null float64
numvotes           147367 non-null float64
_merge             146144 non-null category
studio             3994 non-null object
domestic_gross     3964 non-null float64
foreign_gross      2431 non-null object
year_y             3999 non-null float64
_merge2            147367 non-null category
dtypes: category(2), float64(6), object(4)
memory usage: 12.6+ MB


In [15]:
# Remove NaN titles
imdbom = imdbom.dropna(axis=0,subset=["title"])
imdbom

,title,year_x,runtime_minutes,genres,averagerating,numvotes,_merge,studio,domestic_gross,foreign_gross,year_y,_merge2
0,#1 Serial Killer,2013.0,87.0,Horror,5.6,40.0,both,NaN,NaN,NaN,NaN,left_only
1,#5,2013.0,68.0,"Biography,Comedy,Fantasy",6.8,6.0,both,NaN,NaN,NaN,NaN,left_only
2,#5,2018.0,87.0,Documentary,6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only
3,#50Fathers,2015.0,132.0,Comedy,6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only
4,#66,2015.0,116.0,Action,5.3,18.0,both,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...
147341,ärtico,2014.0,78.0,Drama,6.6,101.0,both,NaN,NaN,NaN,NaN,left_only
147342,è solo questione di punti di vista,2012.0,103.0,"Action,Adventure,Comedy",6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only
147343,è solo questione di tempo,2013.0,72.0,Comedy,6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only
147344,élèctions piege à cons,2014.0,87.0,Comedy,6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only


In [16]:
# Find duplicate titles
duplicates_by(imdbom, "title")

,title,year_x,runtime_minutes,genres,averagerating,numvotes,_merge,studio,domestic_gross,foreign_gross,year_y,_merge2
1,#5,2013.0,68.0,"Biography,Comedy,Fantasy",6.8,6.0,both,NaN,NaN,NaN,NaN,left_only
2,#5,2018.0,87.0,Documentary,6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only
190,(aguirre),2016.0,97.0,"Biography,Documentary",6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only
191,(aguirre),2016.0,98.0,"Biography,Comedy,Documentary",6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only
260,1,2019.0,100.0,Drama,9.2,13.0,both,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...
147185,Éden,2013.0,73.0,Drama,6.1,50.0,both,NaN,NaN,NaN,NaN,left_only
147235,Ódio,2017.0,87.0,Action,6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only
147236,Ódio,2016.0,87.0,Drama,6.5,49.0,left_only,NaN,NaN,NaN,NaN,left_only
147321,Ümmü Sibyan: Zifir,2014.0,87.0,Horror,2.9,257.0,both,NaN,NaN,NaN,NaN,left_only


In [17]:
# Drop duplicates based on same title and year
imdbom = imdbom.drop_duplicates(['title','year_x'], keep ='last')
imdbom = imdbom.drop_duplicates(['title','year_y'], keep ='last')
imdbom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138995 entries, 0 to 147345
Data columns (total 12 columns):
title              138995 non-null object
year_x             137773 non-null float64
runtime_minutes    138995 non-null float64
genres             132683 non-null object
averagerating      138995 non-null float64
numvotes           138995 non-null float64
_merge             137773 non-null category
studio             3381 non-null object
domestic_gross     3358 non-null float64
foreign_gross      2036 non-null object
year_y             3386 non-null float64
_merge2            138995 non-null category
dtypes: category(2), float64(6), object(4)
memory usage: 11.9+ MB


In [18]:
# Combine year_x and year_y columns
imdbom.year_x = imdbom.year_x.combine_first(imdbom.year_y)

In [19]:
imdbom.describe()

,year_x,runtime_minutes,averagerating,numvotes,domestic_gross,year_y
count,138995.000000,138995.000000,138995.000000,1.389950e+05,3.358000e+03,3386.000000
mean,2014.652664,86.348840,6.422999,1.677955e+03,2.875440e+07,2013.959244
std,2.729282,150.727504,1.043520,2.114006e+04,6.699064e+07,2.477573
min,2010.000000,1.000000,1.000000,5.000000e+00,1.000000e+02,2010.000000
25%,2012.000000,75.000000,6.500000,4.900000e+01,1.200000e+05,2012.000000
50%,2015.000000,87.000000,6.500000,4.900000e+01,1.400000e+06,2014.000000
75%,2017.000000,95.000000,6.500000,4.900000e+01,2.790000e+07,2016.000000
max,2115.000000,51420.000000,10.000000,1.841066e+06,9.367000e+08,2018.000000


In [20]:
# Remove unneeded columns and rename year_x
imdbom = imdbom.drop(columns=["year_y", "_merge", "_merge2"]).rename(columns={"year_x":"year"})

In [21]:
imdbom.head(10)

,title,year,runtime_minutes,genres,averagerating,numvotes,studio,domestic_gross,foreign_gross
0,#1 Serial Killer,2013.0,87.0,Horror,5.6,40.0,NaN,NaN,NaN
2,#5,2018.0,87.0,Documentary,6.5,49.0,NaN,NaN,NaN
3,#50Fathers,2015.0,132.0,Comedy,6.5,49.0,NaN,NaN,NaN
4,#66,2015.0,116.0,Action,5.3,18.0,NaN,NaN,NaN
5,#82 Summits,2015.0,50.0,Documentary,6.5,49.0,NaN,NaN,NaN
6,#ALLMYMOVIES,2015.0,87.0,Documentary,8.3,21.0,NaN,NaN,NaN
7,#AbroHilo,2019.0,52.0,Documentary,6.6,8.0,NaN,NaN,NaN
8,#BKKY,2016.0,75.0,Drama,7.4,23.0,NaN,NaN,NaN
9,#Babylon'13,2014.0,87.0,Documentary,6.5,49.0,NaN,NaN,NaN
10,#BeRobin the Movie,2015.0,41.0,Documentary,9.0,31.0,NaN,NaN,NaN


In [22]:
# Select random genre from genres list for new column in which genre is "diaggregated"
import random 
imdbom1 = imdbom[imdbom['genres'].notna()]
imdbom1['genres'].apply(lambda x: random.choice(x.split(",")))

imdbom.head()

,title,year,runtime_minutes,genres,averagerating,numvotes,studio,domestic_gross,foreign_gross
0,#1 Serial Killer,2013.0,87.0,Horror,5.6,40.0,NaN,NaN,NaN
2,#5,2018.0,87.0,Documentary,6.5,49.0,NaN,NaN,NaN
3,#50Fathers,2015.0,132.0,Comedy,6.5,49.0,NaN,NaN,NaN
4,#66,2015.0,116.0,Action,5.3,18.0,NaN,NaN,NaN
5,#82 Summits,2015.0,50.0,Documentary,6.5,49.0,NaN,NaN,NaN


In [23]:
display(imdbom.describe())
display(imdbom.info())
# Need to correct foreign_gross dtype to integer
imdbom["foreign_gross"]=imdbom["foreign_gross"].notnull().astype(float)
display(imdbom.info())

,year,runtime_minutes,averagerating,numvotes,domestic_gross
count,138995.000000,138995.000000,138995.000000,1.389950e+05,3.358000e+03
mean,2014.652664,86.348840,6.422999,1.677955e+03,2.875440e+07
std,2.729282,150.727504,1.043520,2.114006e+04,6.699064e+07
min,2010.000000,1.000000,1.000000,5.000000e+00,1.000000e+02
25%,2012.000000,75.000000,6.500000,4.900000e+01,1.200000e+05
50%,2015.000000,87.000000,6.500000,4.900000e+01,1.400000e+06
75%,2017.000000,95.000000,6.500000,4.900000e+01,2.790000e+07
max,2115.000000,51420.000000,10.000000,1.841066e+06,9.367000e+08


<class 'pandas.core.frame.DataFrame'>
Int64Index: 138995 entries, 0 to 147345
Data columns (total 9 columns):
title              138995 non-null object
year               138995 non-null float64
runtime_minutes    138995 non-null float64
genres             132683 non-null object
averagerating      138995 non-null float64
numvotes           138995 non-null float64
studio             3381 non-null object
domestic_gross     3358 non-null float64
foreign_gross      2036 non-null object
dtypes: float64(5), object(4)
memory usage: 10.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138995 entries, 0 to 147345
Data columns (total 9 columns):
title              138995 non-null object
year               138995 non-null float64
runtime_minutes    138995 non-null float64
genres             132683 non-null object
averagerating      138995 non-null float64
numvotes           138995 non-null float64
studio             3381 non-null object
domestic_gross     3358 non-null float64
foreign_gross      138995 non-null float64
dtypes: float64(6), object(3)
memory usage: 10.6+ MB


None

In [24]:
imdbom.to_csv('imdbom.csv')